In [1]:
# import libriary
import pandas as pd
import os
import sys
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# encoder libirary
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce

# modeling & evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA

# tensorflow
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
## load data
def load_data(path):
    result = pd.read_csv(path)
    return result

file_path = "C:\\Users\\chend\\Documents\\case\\dealer_txns.csv"
# import data
df = load_data(file_path)

In [3]:
df.shape

(278337, 25)

In [4]:
def process_data(df):
    # Create 'PolicyPurchased' column based on 'Returned' column
    df['PolicyPurchased'] = np.where(df['Returned'].isna(), 0, 1)
    
    # Create 'Returned_new' column with filled missing values as 0
    df['Returned_new'] = df['Returned'].fillna(0)
    
    # Convert 'SaleDate' to datetime and extract time information
    df['SaleDate'] = pd.to_datetime(df.SaleDate, format='%y-%m-%d')
    df['SaleYear'] = df['SaleDate'].dt.year
    df['SaleMonth'] = df['SaleDate'].dt.month
    df['SaleDay'] = df['SaleDate'].dt.day
    df['SaleWeekday'] = df['SaleDate'].dt.weekday
    
    # Calculate 'Age' based on 'SaleYear' and 'CarYear'
    df['Age'] = df['SaleYear'] - df['CarYear'] + 1
    
    # Filter rows based on 'Autocheck_score' range
    df = df[(df["Autocheck_score"] >= -5) & (df["Autocheck_score"] <= 5)].copy()
    
    # Create 'IsWeekend' column
    df['IsWeekend'] = df['SaleWeekday'].apply(lambda x: 1 if x >= 5 else 0)

    # Create 'IsOnline' column
    df['IsOnline'] = df.apply(lambda row: 1 if row['OVE'] == 1 or row['Simulcast'] == 1 else 0, axis=1)

    # Create 'NumTransactionsBuyer' column
    df['NumTransactionsBuyer'] = df.groupby('BuyerID')['BuyerID'].transform('count')

    # Create 'CarsBoughtBuyer' column
    buyer_counts = df['BuyerID'].value_counts()
    df['CarsBoughtBuyer'] = df['BuyerID'].map(buyer_counts)

    # Create 'AvgCarAgeByBuyer' column
    df['AvgCarAgeByBuyer'] = df.groupby('BuyerID')['Age'].transform('mean')

    # Create 'price_diff' column
    df['PriceDiff'] = df['SalePrice'] - df['MMR']
    
    # if MMR is missing, fill with Saleprice
    mask = df['MMR'] == 0
    df.loc[mask,'MMR'] = df.loc[mask, 'SalePrice']
        
    return df

In [5]:
df = process_data(df)

In [6]:
df.shape

(266917, 38)

In [7]:
## log tranformation

def create_log_columns(df, columns):
    for column in columns:
        new_column = 'Log' + column
        df[new_column] = np.log1p(df[column])
    return df

log_columns = ['Mileage', 'SalePrice', 'MMR']
df = create_log_columns(df, log_columns)

In [8]:
## mapping conditionreport columns
def apply_mapping_and_convert_to_numeric(df, column, mapping):
    df[column] = df[column].replace(mapping)
    df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

mapping = {"EC": 60, "CL": 50, "AV": 40, "RG": 30, "PR": 20, "SL": 10}
df = apply_mapping_and_convert_to_numeric(df, "ConditionReport", mapping)

In [9]:
### mileage usage per year
def calculate_usage_per_year(row):
    if row['Age'] == 0:
        return row['Mileage']
    else:
        return row['Mileage'] / row['Age']

df['UsagePerYear'] = df.apply(calculate_usage_per_year, axis=1)

In [10]:
# set bin 
def apply_binning(df, column, bins, labels):
    df[column + 'Category'] = pd.cut(df[column], bins=bins, labels=labels)
    return df

bins = [df.LogMileage.min(), 10, 12, df.LogMileage.max()]
labels = ['Low', 'Medium', 'High']
df = apply_binning(df, 'LogMileage', bins, labels)

bins = [df.PriceDiff.min(), -2500, 0, 2500, df.PriceDiff.max()]
labels = ['Much Lower', 'Lower', 'Higher', 'Much Higher']
df = apply_binning(df, 'PriceDiff', bins, labels)

bins = [df['Autocheck_score'].min(), 0, 1, df['Autocheck_score'].max()]
labels = ['Negative', 'Zero', 'Positive']

df = apply_binning(df, 'Autocheck_score', bins, labels)

In [12]:
## drop null values
def drop_nan_values(df, columns):
    df = df.dropna(subset=columns)
    return df

columns_to_dropna = ['PriceDiffCategory', 'Autocheck_scoreCategory', 'LogMileageCategory', 'JDPowersCat']
df = drop_nan_values(df, columns_to_dropna)

In [13]:
## oridinal encoding
def ordinal_encode_columns(df, columns, categories):
    encoder = OrdinalEncoder(categories=categories)
    df[columns] = encoder.fit_transform(df[columns])
    return df

columns_to_encode = ['PriceDiffCategory', 'Autocheck_scoreCategory', 'LogMileageCategory', 'JDPowersCat']
categories = [['Much Lower', 'Lower', 'Higher', 'Much Higher'],
              ['Negative', 'Zero', 'Positive'],
             ['Low', 'Medium', 'High'],
             ['EXCLUDED','FULLSIZE CAR','SPORTS CAR', 'VAN','PICKUP', 'LUXURY CAR', 'COMPACT CAR','MIDSIZE CAR', 'SUV']]

df = ordinal_encode_columns(df, columns_to_encode, categories)#

In [14]:
## create columns 'IsLuxury'
def create_is_luxury_column(df, luxury_brands):
    df['IsLuxury'] = df['CarMake'].apply(lambda x: 1 if x in luxury_brands else 0)
    return df

luxury_brands = ['B M W', 'MERCEDES-BENZ', 'LEXUS', 'CADILLAC', 'LINCOLN', 'ACURA', 'AUDI', 'LAND ROVER', 'PORSCHE', 'MASERATI', 'BENTLEY', 'FERRARI', 'ASTON MARTIN']
df = create_is_luxury_column(df, luxury_brands)

In [15]:
## hashing encoding

def hashEncoded(df, columns):
    for column in columns:
        # Initialize HashingEncoder
        encoder = ce.HashingEncoder(n_components=8)

        # Fit and transform the column and replace it
        encoded = encoder.fit_transform(df[column])
        encoded.columns = f'{column}_' + encoded.columns

        # Join the encoded dataframe with the original dataframe
        df = df.join(encoded)

    return df


columns = ['SellingLocation', 'CarMake']
df = hashEncoded(df, columns)

In [16]:
## conditionreport column is critical
## so use exist value to predict null value
## Decision tree regressor

def predict_missing_condition_report(df):
    # Selecting columns that have correlation greater than 0.3 with 'ConditionReport'
    corr = df.corr()
    selected_columns = corr[abs(corr['ConditionReport']) > 0.3].index

    # Dropping 'ConditionReport' from selected_columns
    selected_columns = selected_columns.drop('ConditionReport')

    # Creating new DataFrame with non-null 'ConditionReport' rows
    df_notnull = df[df['ConditionReport'].notnull()]

    # Splitting data into train and test
    X = df_notnull[selected_columns]
    y = df_notnull['ConditionReport']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Initializing and fitting decision tree regressor
    tree = DecisionTreeRegressor(random_state=42)
    tree.fit(X_train, y_train)

    # Predicting 'ConditionReport' for test data and calculating RMSE
    y_pred = tree.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error: {rmse}")

    # Now predicting 'ConditionReport' for rows with null values
    df_null = df[df['ConditionReport'].isnull()]
    X_null = df_null[selected_columns]
    predicted_condition_report = tree.predict(X_null)

    # Filling null values in 'ConditionReport' with predicted values
    df.loc[df['ConditionReport'].isnull(), 'ConditionReport'] = predicted_condition_report
    
    return df

df = predict_missing_condition_report(df)

Root Mean Squared Error: 9.228086042146874


In [17]:
def create_interaction_features(df):
    # # price difference percentage
    df['PriceDiffPercentage'] = abs(df['PriceDiff'] / df['MMR']) * 100
    df['Buyer_Total_Arbitrated'] = df.groupby('BuyerID')['Arbitrated'].transform('sum')
    df['Buyer_Average_Arbitrated'] = df.groupby('BuyerID')['Arbitrated'].transform('mean')

    df['Buyer_Total_Salvage'] = df.groupby('BuyerID')['Salvage'].transform('sum')
    df['Buyer_Average_Salvage'] = df.groupby('BuyerID')['Salvage'].transform('mean')

    # Binary features
    for feature in ['PSIEligible', 'DSEligible', 'IsLuxury']:
        df[f'Buyer_Total_{feature}'] = df.groupby('BuyerID')[feature].transform('sum')
        df[f'Buyer_Average_{feature}'] = df.groupby('BuyerID')[feature].transform('mean')

    # Continuous features
    for feature in ['Autocheck_score', 'UsagePerYear', 'PriceDiffPercentage', 'LogMileageCategory', 'LogSalePrice']:
        df[f'Buyer_Total_{feature}'] = df.groupby('BuyerID')[feature].transform('sum')
        df[f'Buyer_Average_{feature}'] = df.groupby('BuyerID')[feature].transform('mean')

    # Categorical features
    for feature in ['SaleMonth', 'JDPowersCat']:
        mode = lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else np.nan
        df[f'Buyer_Mode_{feature}'] = df.groupby('BuyerID')[feature].transform(mode)

    # Binary features
    for feature in ['Salvage', 'PSIEligible', 'DSEligible', 'IsLuxury']:
        df[f'Arbitrated_{feature}'] = df['Arbitrated'] * df[feature]

    # Continuous features
    for feature in ['UsagePerYear', 'LogMileageCategory', 'LogSalePrice']:
        df[f'Arbitrated_{feature}'] = df['Arbitrated'] * df[feature]

    # Categorical features
    mode = lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else np.nan
    for feature in ['SaleMonth', 'JDPowersCat']:
        df[f'Arbitrated_{feature}'] = df.groupby('Arbitrated')[feature].transform(mode)

    df['Buyer_Seller_Price'] = df.groupby(['BuyerID', 'SellerID'])['LogSalePrice'].transform('mean')
    df['Buyer_Arbitrated_Transactions'] = df.groupby(['BuyerID', 'Arbitrated'])['NumTransactionsBuyer'].transform('sum')
    df['Buyer_Autocheck_Price'] = df.groupby(['BuyerID', 'Autocheck_score'])['LogSalePrice'].transform('mean')
    
    return df

df = create_interaction_features(df)

In [18]:
def create_month_interaction_features(df):
    mode = lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else np.nan
    df['Buyer_Make0_Month'] = df.groupby(['BuyerID', 'CarMake_col_0'])['SaleMonth'].transform(mode)
    df['Buyer_Make1_Month'] = df.groupby(['BuyerID', 'CarMake_col_1'])['SaleMonth'].transform(mode)
    df['Buyer_Make2_Month'] = df.groupby(['BuyerID', 'CarMake_col_2'])['SaleMonth'].transform(mode)
    df['Buyer_Make3_Month'] = df.groupby(['BuyerID', 'CarMake_col_3'])['SaleMonth'].transform(mode)
    df['Buyer_Make4_Month'] = df.groupby(['BuyerID', 'CarMake_col_4'])['SaleMonth'].transform(mode)
    df['Buyer_Make5_Month'] = df.groupby(['BuyerID', 'CarMake_col_5'])['SaleMonth'].transform(mode)
    df['Buyer_Make6_Month'] = df.groupby(['BuyerID', 'CarMake_col_6'])['SaleMonth'].transform(mode)
    df['Buyer_Make7_Month'] = df.groupby(['BuyerID', 'CarMake_col_7'])['SaleMonth'].transform(mode)
    return df

df = create_month_interaction_features(df)

In [19]:
df.shape

(265980, 105)

In [20]:
## drop columns not explain the case 

def drop_columns(df, columns):
    df = df.drop(columns, axis=1)
    return df

columns_to_drop = ['VIN', 'SaleYear', 'Returned', 'SellingLocation', 'CarMake', 'CarYear', 'SaleDate', 'Mileage', 'SalePrice', 'MMR']
df = drop_columns(df, columns_to_drop)

In [21]:
def pca_var(df):
    # Select the features for PCA
    X = df.drop('Returned_new', axis=1)  # Drop the target column
    y = df['Returned_new']  # Target column

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Perform PCA
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    
    var_exp = pca.explained_variance_ratio_
    print(f'Explained variance (first 30 components): {np.round(var_exp[:30],3)}')
    
    # Generate the cumulative explained variance.
    cum_var_exp = np.cumsum(var_exp)
    print(f'Cumulative explained variance (first 30 components): {np.round(cum_var_exp[:30],3)}')

pca_var(df)

Explained variance (first 30 components): [0.158 0.101 0.075 0.063 0.031 0.03  0.024 0.023 0.02  0.019 0.018 0.017
 0.015 0.015 0.014 0.014 0.013 0.013 0.013 0.013 0.012 0.012 0.012 0.012
 0.012 0.011 0.011 0.011 0.011 0.011]
Cumulative explained variance (first 30 components): [0.158 0.259 0.335 0.398 0.43  0.459 0.483 0.505 0.525 0.544 0.562 0.58
 0.595 0.61  0.624 0.638 0.652 0.664 0.677 0.69  0.702 0.714 0.726 0.738
 0.749 0.761 0.772 0.783 0.794 0.805]


In [22]:
def apply_pca(df, n_components):
    # Select the features for PCA
    X = df.drop('Returned_new', axis=1)  # Drop the target column

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Perform PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)

    # Create a new DataFrame with the reduced dimensions
    df_reduced = pd.DataFrame(data=X_pca, columns=[f"PC{i+1}" for i in range(n_components)])

    return df_reduced


df_reduced = apply_pca(df, 30)

In [23]:
df_reduced.isnull().sum()

PC1     0
PC2     0
PC3     0
PC4     0
PC5     0
PC6     0
PC7     0
PC8     0
PC9     0
PC10    0
PC11    0
PC12    0
PC13    0
PC14    0
PC15    0
PC16    0
PC17    0
PC18    0
PC19    0
PC20    0
PC21    0
PC22    0
PC23    0
PC24    0
PC25    0
PC26    0
PC27    0
PC28    0
PC29    0
PC30    0
dtype: int64

In [24]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_reduced, df['Returned_new'], test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

Recall: 0.6998769987699877


In [25]:
## dataset us

df_new = df[['PolicyPurchased',
 'Arbitrated_UsagePerYear',
 'Arbitrated_LogSalePrice',
 'Arbitrated',
 'PSI',
 'Arbitrated_DSEligible',
 'SaleMonth',
 'Arbitrated_LogMileageCategory',
 'LIGHTR',
 'SellerID',
 'Buyer_Average_Arbitrated',
 'LogMileage',
 'Buyer_Seller_Price',
 'PriceDiffPercentage',
 'LogSalePrice',
 'Arbitrated_PSIEligible',
 'LogMMR',
 'Autocheck_score',
 'UsagePerYear',
 'Buyer_Autocheck_Price',
 'PriceDiff',
 'Buyer_Arbitrated_Transactions',
 'SaleDay',
 'ConditionReport',
 'SaleWeekday',
 'Buyer_Total_Arbitrated',
 'Age',
 'Buyer_Total_LogSalePrice',
 'Buyer_Total_Autocheck_score',
 'CarsBoughtBuyer', 'Returned_new']]


In [27]:
df_new.Returned_new.value_counts()

0.0    261871
1.0      4109
Name: Returned_new, dtype: int64